This is to help work through some plex issues.

In [2]:
#!pwsh

$cluster = "admin@equestria"
$namespace = "equestria"
$pvcName = "plex"
$podName = "plex-debug"

kubectl config use-context $cluster

Switched to context "admin@equestria".


In [7]:
#!pwsh

$debugPodYaml = @"
apiVersion: v1
kind: Pod
metadata:
  name: $podName
  namespace: $namespace
  labels:
    app: plex-debug
spec:
  restartPolicy: Never
  securityContext:
    runAsUser: 568
    runAsGroup: 568
    fsGroup: 568
    fsGroupChangePolicy: OnRootMismatch
    supplementalGroups: [44, 109, 303, 568, 10000]
  containers:
  - name: toolbox
    image: 1.25.1-bookworm@sha256:c4bc0741e3c79c0e2d47ca2505a06f5f2a44682ada94e1dba251a3854e60c2bd
    command: ["/bin/bash"]
    args:
      - -c
      - |
        echo "🚀 Debug pod ready. PVC mounted at /config"
        echo "📁 Contents:"
        ls -lah /config
        echo ""
        echo "💤 Keeping pod alive. Use 'kubectl exec' to access."
        trap : TERM INT; sleep infinity & wait
    securityContext:
      allowPrivilegeEscalation: true
    volumeMounts:
    - name: plex-data
      mountPath: /config
  volumes:
  - name: plex-data
    persistentVolumeClaim:
      claimName: $pvcName
"@

Write-Host "🚀 Creating debug pod..." -ForegroundColor Cyan
$debugPodYaml | kubectl apply -f -

Write-Host "`n⏳ Waiting for pod to be ready..."
kubectl wait --for=condition=Ready pod/$podName -n $namespace --timeout=120s

Write-Host "`n✅ Debug pod ready!" -ForegroundColor Green
kubectl get pod $podName -n $namespace

Write-Host "🔧 Opening shell in debug pod..." -ForegroundColor Cyan
Write-Host "Plex config is at: /config" -ForegroundColor Yellow

🚀 Creating debug pod...

⏳ Waiting for pod to be ready...

✅ Debug pod ready!
🔧 Opening shell in debug pod...
Plex config is at: /config
pod/plex-debug configured
pod/plex-debug condition met
NAME         READY   STATUS    RESTARTS   AGE
plex-debug   1/1     Running   0          3m30s


In [2]:
kubectl exec -it $podName -n $namespace -- ls -al "/config/Plug-in Support/Databases"


Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.RemoteException: Unable to use a TTY - input is not a terminal or the right kind of file SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.ParentContainsErrorRecordException: Unable to use a TTY - input is not a terminal or the right kind of file Message Unable to use a TTY - input is not a terminal or the right kind of file TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Unable to use a TTY - input is not a terminal or the right kind of file Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Unable to use a TTY - input is not a terminal or the right kind of file CategoryInfo NotSpecified: (Unable to use a TTY… right kind of file:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Unable to use a TTY - input is not a terminal or the right kind of file TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Managem

total 5039892
drwxrwsr-x. 2 568 568      4096 Oct 10 01:05 .
drwxrwsr-x. 7 568 568      4096 Jul 31 11:58 ..
-rwxr-xr-x. 1 568 568 712108032 Oct 10 01:04 com.plexapp.plugins.library.blobs.db
-rw-r--r--. 1 568 568 658400256 Sep 27 06:00 com.plexapp.plugins.library.blobs.db-2025-09-27
-rw-r--r--. 1 568 568 659464192 Sep 30 06:02 com.plexapp.plugins.library.blobs.db-2025-09-30
-rw-r--r--. 1 568 568 712108032 Oct  3 06:04 com.plexapp.plugins.library.blobs.db-2025-10-03
-rw-r--r--. 1 568 568 712390656 Oct  9 06:02 com.plexapp.plugins.library.blobs.db-2025-10-09
-rwxr-xr-x. 1 568 568 350263296 Oct 10 01:05 com.plexapp.plugins.library.db
-rw-r--r--. 1 568 568 336051200 Sep 27 06:00 com.plexapp.plugins.library.db-2025-09-27
-rw-r--r--. 1 568 568 340588544 Sep 30 06:02 com.plexapp.plugins.library.db-2025-09-30
-rwxr-xr-x. 1 568 568 339700736 Oct  3 06:03 com.plexapp.plugins.library.db-2025-10-03
-rw-r--r--. 1 568 568 339708928 Oct  9 06:02 com.plexapp.plugins.library.db-2025-10-09
-rwxr-xr-x. 1

In [3]:
#!pwsh

Write-Host "⚠️  Scaling down Plex..." -ForegroundColor Yellow
kubectl scale helmrelease plex -n $namespace --replicas=0

Write-Host "`nWaiting for Plex pod to terminate..."
kubectl wait --for=delete pod -n $namespace -l app.kubernetes.io/name=plex --timeout=120s

Write-Host "✅ Plex scaled down" -ForegroundColor Green


⚠️  Scaling down Plex...

Waiting for Plex pod to terminate...
✅ Plex scaled down


Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found Exception System.Management.Automation.RemoteException: Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found Exception System.Management.Automation.ParentContainsErrorRecordException: Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found Message Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found CategoryInfo NotSpecified: (Error from server (…io "plex" not found:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fba

error: timed out waiting for the condition on pods/plex-78cd8c67b7-x9hwp Exception System.Management.Automation.RemoteException: error: timed out waiting for the condition on pods/plex-78cd8c67b7-x9hwp SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord error: timed out waiting for the condition on pods/plex-78cd8c67b7-x9hwp Exception System.Management.Automation.ParentContainsErrorRecordException: error: timed out waiting for the condition on pods/plex-78cd8c67b7-x9hwp Message error: timed out waiting for the condition on pods/plex-78cd8c67b7-x9hwp TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message error: timed out waiting for the condition on pods/plex-78cd8c67b7-x9hwp Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject error: timed out waiting for the condition on pods/plex-78cd8c67b7-x9hwp CategoryInfo NotSpecified: (error: timed out wa…ex-78cd8c67b7-x9hwp:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName error: timed out waiting for the condition on pods/plex-78cd8c67b7-x9hwp TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System

In [ ]:
#!pwsh

$backupDate = "2025-10-03"

kubectl exec -it $podName -n $namespace -- cp "/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-$($backupDate)" "/config/Plug-in Support/Databases/com.plexapp.plugins.library.db"
kubectl exec -it $podName -n $namespace -- cp "/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-$($backupDate)" "/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db"
kubectl exec -it $podName -n $namespace -- find "/config/Plug-in Support/Databases/" -name "*-shm" -o -name "*-wal" | xargs rm -f

Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.RemoteException: Unable to use a TTY - input is not a terminal or the right kind of file SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.ParentContainsErrorRecordException: Unable to use a TTY - input is not a terminal or the right kind of file Message Unable to use a TTY - input is not a terminal or the right kind of file TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Unable to use a TTY - input is not a terminal or the right kind of file Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Unable to use a TTY - input is not a terminal or the right kind of file CategoryInfo NotSpecified: (Unable to use a TTY… right kind of file:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Unable to use a TTY - input is not a terminal or the right kind of file TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Managem

Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.RemoteException: Unable to use a TTY - input is not a terminal or the right kind of file SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.ParentContainsErrorRecordException: Unable to use a TTY - input is not a terminal or the right kind of file Message Unable to use a TTY - input is not a terminal or the right kind of file TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Unable to use a TTY - input is not a terminal or the right kind of file Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Unable to use a TTY - input is not a terminal or the right kind of file CategoryInfo NotSpecified: (Unable to use a TTY… right kind of file:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Unable to use a TTY - input is not a terminal or the right kind of file TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Managem

Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.RemoteException: Unable to use a TTY - input is not a terminal or the right kind of file SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.ParentContainsErrorRecordException: Unable to use a TTY - input is not a terminal or the right kind of file Message Unable to use a TTY - input is not a terminal or the right kind of file TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Unable to use a TTY - input is not a terminal or the right kind of file Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Unable to use a TTY - input is not a terminal or the right kind of file CategoryInfo NotSpecified: (Unable to use a TTY… right kind of file:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Unable to use a TTY - input is not a terminal or the right kind of file TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Managem

In [ ]:
#!pwsh

Write-Host "🔄 Scaling Plex back up..." -ForegroundColor Cyan
kubectl scale deployment plex -n $namespace --replicas=1

Write-Host "`n⏳ Waiting for Plex to be ready..."
Start-Sleep -Seconds 5
kubectl wait --for=condition=Ready pod -n $namespace -l app.kubernetes.io/name=plex --timeout=300s

Write-Host "`n✅ Plex is back up!" -ForegroundColor Green
kubectl get pods -n $namespace -l app.kubernetes.io/name=plex

🔄 Scaling Plex back up...

⏳ Waiting for Plex to be ready...

✅ Plex is back up!


Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found Exception System.Management.Automation.RemoteException: Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found Exception System.Management.Automation.ParentContainsErrorRecordException: Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found Message Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found CategoryInfo NotSpecified: (Error from server (…io "plex" not found:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Error from server (NotFound): helmreleases.helm.toolkit.fluxcd.io "plex" not found TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 92 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fba

pod/plex-78cd8c67b7-x9hwp condition met
NAME                    READY   STATUS    RESTARTS         AGE
plex-78cd8c67b7-x9hwp   1/1     Running   10 (6m24s ago)   27m


In [5]:
#!pwsh

Write-Host "🧹 Cleaning up debug pod..." -ForegroundColor Yellow
kubectl delete pod $podName -n $namespace --wait=true

Write-Host "✅ Debug pod deleted" -ForegroundColor Green

🧹 Cleaning up debug pod...
✅ Debug pod deleted
pod "plex-debug" deleted from equestria namespace
